# Выбор локации для скважины

Допустим, вы работаете в нефтедобывающей компании. Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

pd.options.mode.chained_assignment = None

In [21]:
df = [0] * 3
for i in range(0,3):
    df[i] = pd.read_csv(f'/datasets/geo_data_{i}.csv')

По условию во всех датасетах значения столбца `id` - это уникальный идентификатор скважины.  
Проверим, если в этом столбце есть дубликаты, удалим их.

In [22]:
for i in range(0,3):
    print(f'Дубликатов в {i} датасете - {df[i]["id"].duplicated().sum()}, всего строк {df[i]["id"].count()}')
    df[i].drop_duplicates(subset=['id'], inplace=True)
    print(f'Дубликатов в {i} датасете - {df[i]["id"].duplicated().sum()}, всего строк {df[i]["id"].count()}')

Дубликатов в 0 датасете - 10, всего строк 100000
Дубликатов в 0 датасете - 0, всего строк 99990
Дубликатов в 1 датасете - 4, всего строк 100000
Дубликатов в 1 датасете - 0, всего строк 99996
Дубликатов в 2 датасете - 4, всего строк 100000
Дубликатов в 2 датасете - 0, всего строк 99996


**Вывод:**  
Данные подготовлены, дубликаты удалены.


In [23]:
for i in range(0,3):
    print(f'Пропущенных значений в {i} датасете - {df[i].isna().sum().sum()}')

Пропущенных значений в 0 датасете - 0
Пропущенных значений в 1 датасете - 0
Пропущенных значений в 2 датасете - 0


Пропущенных значений нет

In [24]:
for i in range(0,3):
        print(df[i].describe())

                 f0            f1            f2       product
count  99990.000000  99990.000000  99990.000000  99990.000000
mean       0.500454      0.250141      2.502629     92.499684
std        0.871844      0.504430      3.248149     44.288304
min       -1.408605     -0.848218    -12.088328      0.000000
25%       -0.072572     -0.200877      0.287784     56.497069
50%        0.502405      0.250252      2.515969     91.847928
75%        1.073626      0.700640      4.715035    128.563699
max        2.362331      1.343769     16.003790    185.364347
                 f0            f1            f2       product
count  99996.000000  99996.000000  99996.000000  99996.000000
mean       1.141209     -4.796608      2.494501     68.823916
std        8.965815      5.119906      1.703579     45.944663
min      -31.609576    -26.358598     -0.018144      0.000000
25%       -6.298551     -8.267985      1.000021     26.953261
50%        1.153055     -4.813172      2.011475     57.085625
75%     

Среднее значение количества сырья в 1 датасете ниже чем в остальных, и 25% скважин имеют запасы меньше 27 тыс.бар - в этом регионе запасы откровенно меньше, чем в остальных.  
В 0 и 2 датасете 25% скважин имеют запасы более 128 тыс бар - богатые регионы.  
**Признаки f0, f1, f2 имеют разный размер - необходимо масштабировать.**

Проверим корреляцию между признаками и целью

In [25]:
for i in range(0,3):
    print('Датасет - ', i)
    for n in ['f0', 'f1', 'f2']:
        print(f'Корреляция между {n} и product равна {df[i][n].corr(df[i]["product"])}')
    print()

Датасет -  0
Корреляция между f0 и product равна 0.14350354470629628
Корреляция между f1 и product равна -0.19235060940995077
Корреляция между f2 и product равна 0.483630982761089

Датасет -  1
Корреляция между f0 и product равна -0.030522920755861983
Корреляция между f1 и product равна -0.010153692790654676
Корреляция между f2 и product равна 0.9993969736591198

Датасет -  2
Корреляция между f0 и product равна -0.001977658242440491
Корреляция между f1 и product равна -0.0010545826382116158
Корреляция между f2 и product равна 0.44586699434895694



Корреляции между признаками f0, f1 и product нет.  
Корреляция между признаком f2 и product в датасетах 0 и 2 - есть, но слабая.  
Корреляция между признаком f2 и product в датасете 1 - есть, очень высокая, эти величины взаимосвязанны.    
По сути f2 в датасете 1 это product в других величинах. 

**Вывод**  
Из данных удалены дубликаты.   
В данных нет пропусков.  
Установлена необходимость масштабирования.  
Выявлена высокая корреляция одного признака с целью в одном датасете.

## Обучение и проверка модели

Разделим выборки

In [26]:
features_train = [0] * 3
features_valid = [0] * 3
target_train = [0] * 3
target_valid = [0] * 3
prediction_valid = [0] * 3

for i in range(3):
    features = df[i].drop(['id', 'product'], axis=1)
    target = df[i]['product']
    features_train[i], features_valid[i], target_train[i], target_valid[i] = \
        train_test_split(features, target, test_size=0.25, random_state=54321)

Проверим размер выборок

In [27]:
for i in range(3):
    print(features_train[i].shape, target_train[i].shape, features_valid[i].shape, target_valid[i].shape)

(74992, 3) (74992,) (24998, 3) (24998,)
(74997, 3) (74997,) (24999, 3) (24999,)
(74997, 3) (74997,) (24999, 3) (24999,)


Масштабируем признаки

In [28]:
numeric = ['f0', 'f1', 'f2']
for i in range(3):
    scaler = StandardScaler()
    scaler.fit(features_train[i][numeric])
    features_train[i][numeric] = scaler.transform(features_train[i][numeric])
    features_valid[i][numeric] = scaler.transform(features_valid[i][numeric])

Обучим модель

In [29]:
for i in range(3):
    model = LinearRegression()
    model.fit(features_train[i], target_train[i])
    prediction_valid[i] = model.predict(features_valid[i])

Рассчитаем средний запас предсказанного сырья и RMSE модели.

In [30]:
for i in range(3):
    print(f'Данные по {i} региону')
    print('Средний запас =', prediction_valid[i].mean())
    print('RMSE =', mean_squared_error(target_valid[i], prediction_valid[i]) ** 0.5)
    print()

Данные по 0 региону
Средний запас = 92.48003680900305
RMSE = 37.640122873348126

Данные по 1 региону
Средний запас = 68.4953539574733
RMSE = 0.8932765122005417

Данные по 2 региону
Средний запас = 94.85117513247839
RMSE = 40.28905957282013



**Вывод:**  
Согласно расчетам самые большие средние запасы имеет 2 регион.  
В 0 и 2 регионе значение средней абсолютной ошибки достаточно велико, это значит что в данных много разных значений сильно отличающихся друг от друга.  
В тоже время в 1 регионе ошибка очень маленькая, это значит что в данных много очень близких значений.

## Подготовка к расчёту прибыли

Рассчитаем какой запас сырья необходим для безубыточной разработки одной скважины.

In [31]:
BUDGET = 10_000 # Бюджет на регион (на 200 скважин), в млн. руб
REVENUE_PER = 0.45 # Доход с 1 единицы сырья, в млн. руб
ALPHA = 0.025 # Порог вероятности
WELL_ALL = 500 # Количество исследуемых скважин в регионе
WELL_GOOD = 200 # Количество разрабатываемых скважин в регионе
BUDGET_PER_WELL = BUDGET / WELL_GOOD # Бюджет на 1 скважину
PRODUCT_PER_WELL = BUDGET_PER_WELL / REVENUE_PER # 111.11 Необходимый запас сырья на одну скважину для безубыточной разработки

In [32]:
print(f'Для безубыточной разработки скважины нужен запас сырья равный {round(PRODUCT_PER_WELL, 2)} тыс.баррелей.')

Для безубыточной разработки скважины нужен запас сырья равный 111.11 тыс.баррелей.


Средние запасы по регионам

In [33]:
for i in range(3):
    print(f'Данные по {i} региону')
    print('Средний запас =', prediction_valid[i].mean())
    print()

Данные по 0 региону
Средний запас = 92.48003680900305

Данные по 1 региону
Средний запас = 68.4953539574733

Данные по 2 региону
Средний запас = 94.85117513247839



**Вывод:**  
Средний запас в каждом регионе меньше необходимого для безубыточной добычи. Наугад бурить скважины нельзя, будут убытки.

## Расчёт прибыли и рисков 

Отберем 200 лучших скважин по предсказаниям в каждом регионе и посчитаем реальные запасы сырья в них.  

Сохраним первоначальный индекс в датасетах, что бы при необходимости можно было идентифицировать нужные скважины.

In [34]:
def revenue(top):
    product = top.sum()
    revenue = product * REVENUE_PER
    profit = product * REVENUE_PER - BUDGET
    return int(product), int(revenue), int(profit)

In [35]:
target_df = [0] * 3
for i in range(3):
    target_df[i] = pd.concat([target_valid[i],\
                             pd.Series(prediction_valid[i], name='predict_product', index=target_valid[i].index)], axis=1)                                        
    prod_top, revenue_top, profit_top = \
                            revenue(target_df[i].sort_values(by='predict_product', ascending=False)['product'][:WELL_GOOD])
    print(f'В {i} регионе:')
    print(f'Запасы сырья в 200-х лучших скважин - {prod_top} тыс.баррелей')
    print(f'Выручка в 200-х лучших скважин - {revenue_top} млн. руб')
    print(f'Прибыль в 200-х лучших скважин - {profit_top} млн. руб')
    print()

В 0 регионе:
Запасы сырья в 200-х лучших скважин - 29522 тыс.баррелей
Выручка в 200-х лучших скважин - 13285 млн. руб
Прибыль в 200-х лучших скважин - 3285 млн. руб

В 1 регионе:
Запасы сырья в 200-х лучших скважин - 27589 тыс.баррелей
Выручка в 200-х лучших скважин - 12415 млн. руб
Прибыль в 200-х лучших скважин - 2415 млн. руб

В 2 регионе:
Запасы сырья в 200-х лучших скважин - 27863 тыс.баррелей
Выручка в 200-х лучших скважин - 12538 млн. руб
Прибыль в 200-х лучших скважин - 2538 млн. руб



**Вывод:**  
Несмотря на то, что средние запасы по каждому регионы ниже необходимых для разработки и получения прибыли, в каждом регионе можно отобрать 200 скважин, которые будут рентабельными.  
Пока самым предпочтительным выглядит 0 регион.  

## Риски и прибыль для каждого региона

Найдем распределение прибыли, среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [36]:
state = np.random.RandomState(12345)
for i in range(3):
    values = []
    for b in range(1000):
        top = target_df[i].sample(n=WELL_ALL, replace=True, random_state=state)
        top = top.sort_values(by='predict_product', ascending=False)[:WELL_GOOD]
        values.append(revenue(top['product'])[2])
    values = pd.Series(values)
    loss = round(values.loc[values < 0].count()/len(values)*100, 1)
    print(f'Регион {i}')
    print(f'Средняя прибыль равна {values.mean()}')
    print(f'95%-й доверительный интервал с {values.quantile(0.025)} до {values.quantile(0.975)}')
    print(f'Риск убытков равен {loss} %')
    print()

Регион 0
Средняя прибыль равна 426.699
95%-й доверительный интервал с -46.074999999999996 до 906.05
Риск убытков равен 4.5 %

Регион 1
Средняя прибыль равна 407.925
95%-й доверительный интервал с 25.900000000000006 до 808.0999999999999
Риск убытков равен 1.9 %

Регион 2
Средняя прибыль равна 353.847
95%-й доверительный интервал с -200.14999999999998 до 845.2249999999998
Риск убытков равен 9.6 %



**Вывод**   
    
Для разработки скважин подходит **только** 1 регион.   
- риск убытков 1.9%  
- значение средней прибыли - не низкое  
- все значения в доверительном интервале положительные  
    
В 0 регионе риск убытков 4.5%, во 2 регионе 9.6%. Это больше 2.5% подходящих по условию.  
Во 2 регионе очень большие отрицательные значения в доверительном интервале, значит разработка может быть очень убыточной, и скважины сильно различаются по количеству запасов сырья.  
0 регион богаче по запасом по сравнению со вторым, но все равно риски велики.

Во время выполнения проекта, я выполнил следующие шаги.
- загрузил данные
- удалил дубликаты
- разбил на выборки
- масштабировал признаки
- обучил модель линейной регресси и сделал предсказания
- посчитал средние запасы
- из предсказаний отобрал 200 лучших для каждого региона
- посчитал реальные запасы и прибыль в отобранных скважинах
- сделал 1000 выборок по 500 скважин из каждого региона
- отобрал 200 лучших из них и посчитал прибыль
- посчитал среднее значение прибыли и доверительный 95% интервал для каждого региона

